### Loading and viewing the dataset

Here we will be seeing the dataset that we are going to use, as well as the intuition of all the operations we will be performing on the main code. This is just a forked branch from the main path in section 1b.

In [1]:
with open('cleaned_dataset.txt', 'r') as f:
    text = f.read()

data = text[:1000]
print(data[:100])

M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly norm
